<a href="https://colab.research.google.com/github/ykato27/BERT-Japanese/blob/main/4_BERT_livedoor_news_IIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 日本語BERTでlivedoorニュースを相互情報量最大化(IIC)でクラスタリング

In [1]:
# 乱数シードの固定

import os
import random
import numpy as np
import torch

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合


### GPUの使用可能を確認

画面上部のメニュー ランタイム > ランタイムのタイプを変更 で、 ノートブックの設定 を開く

ハードウェアアクセラレータに GPU を選択し、 保存 する

In [2]:
# GPUの使用確認：True or False
torch.cuda.is_available()

# TrueならGPU使用可能

True

## 準備1：livedoorニュースをダウンロードして、PyTorchのDataLoaderに変換


In [3]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2021-07-03 06:25:07--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz.4’

ldcc-20140209.tar.g 100%[===================>]   8.44M  5.00MB/s    in 1.7s    

2021-07-03 06:25:09 (5.00 MB/s) - ‘ldcc-20140209.tar.gz.4’ saved [8855190/8855190]



In [4]:
# ファイルを解凍し、カテゴリー数と内容を確認
import tarfile
import os

# 解凍
tar = tarfile.open("ldcc-20140209.tar.gz", "r:gz")
tar.extractall("./data/livedoor/")
tar.close()

# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("./data/livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)


['CHANGES.txt', 'livedoor-homme', 'it-life-hack', 'topic-news', 'movie-enter', 'smax', 'sports-watch', 'dokujo-tsushin', 'peachy', 'README.txt', 'kaden-channel']
カテゴリー数: 9
['livedoor-homme', 'it-life-hack', 'topic-news', 'movie-enter', 'smax', 'sports-watch', 'dokujo-tsushin', 'peachy', 'kaden-channel']


In [5]:
# 本文を取得する前処理関数を定義


def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text


In [6]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)


In [7]:
# pandasのDataFrameにする
import pandas as pd

df = pd.DataFrame({'text': list_text, 'label': list_label})

# 大きさを確認しておく（7,376文章が存在）
print(df.shape)


(7376, 2)


In [8]:
# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

# DataFrameにカテゴリーindexの列を作成
df["label_index"] = df["label"].map(dic_cat2id)

# label列を消去し、text, indexの順番にする
df = df.loc[:, ["text", "label_index"]]


In [9]:
# 順番をシャッフルする
df = df.sample(frac=1, random_state=123).reset_index(drop=True)


In [10]:
# tsvファイルで保存する

# 全体の2割の文章数
len_0_2 = len(df) // 5

# 前から2割をテストデータとする
df[:len_0_2].to_csv("./test.tsv", sep='\t', index=False, header=None)
print(df[:len_0_2].shape)

# 前2割からを訓練&検証データとする
df[len_0_2:].to_csv("./train_eval.tsv", sep='\t', index=False, header=None)
print(df[len_0_2:].shape)


(1475, 2)
(5901, 2)


In [11]:
# tsvファイルをダウンロードしたい場合
from google.colab import files

# ダウンロードする場合はコメントを外す
# 少し時間がかかる（4MB）
# files.download("./test.tsv")


# ダウンロードする場合はコメントを外す
# 少し時間がかかる（18MB）
# files.download("./train_eval.tsv")


- LivedoorニュースをBERT用のDataLoaderにする
- Hugginfaceのリポジトリの案内とは異なり、torchtextを使用した手法で実装

In [12]:
# MeCabとtransformersの用意
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install transformers==2.9.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.8)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.13)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already instal

In [13]:
!sudo cp /etc/mecabrc /usr/local/etc/

In [14]:
import torch
import torchtext  # torchtextを使用
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

# 日本語BERTの分かち書き用tokenizerです
tokenizer = BertJapaneseTokenizer.from_pretrained(
    'bert-base-japanese-whole-word-masking')


In [15]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します

max_length = 512  # 東北大学_日本語版の最大の単語数（サブワード数）は512


def tokenizer_512(input_text):
    """torchtextのtokenizerとして扱えるように、512単語のpytorchでのencodeを定義。ここで[0]を指定し忘れないように"""
    return tokenizer.encode(input_text, max_length=512, return_tensors='pt')[0]


TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, fix_length=max_length, pad_token=0)
# 注意：tokenize=tokenizer.encodeと、.encodeをつけます。padding[PAD]のindexが0なので、0を指定します。

LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

# (注釈)：各引数を再確認
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリーに追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を用意するかどうか
# fix_length：全部の文章をfix_lengthと同じ長さになるように、paddingします
# init_token, eos_token, pad_token, unk_token：文頭、文末、padding、未知語に対して、どんな単語を与えるかを指定


In [16]:
# 各tsvファイルを読み込み、分かち書きをしてdatasetにします
# 少し時間がかかります
# train_eval：5901個、test：1475個
dataset_train_eval, dataset_test = torchtext.legacy.data.TabularDataset.splits(
    path='.', train='train_eval.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])



In [17]:
# torchtext.data.Datasetのsplit関数で訓練データと検証データを分ける
# train_eval：5901個、test：1475個

dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio=1.0 - 1475/5901, random_state=random.seed(1234))

# datasetの長さを確認してみる
print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())


4426
1475
1475


In [18]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
batch_size = 16  # BERTでは16、32あたりを使用する

dl_train = torchtext.legacy.data.Iterator(
    dataset_train, batch_size=batch_size, train=True)

dl_eval = torchtext.legacy.data.Iterator(
    dataset_eval, batch_size=batch_size, train=False, sort=False)

dl_test = torchtext.legacy.data.Iterator(
    dataset_test, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dl_train, "val": dl_eval}


## 準備2：BERTでlivedoorニュースの記事をベクトル化する

In [19]:
from transformers.modeling_bert import BertModel

# BERTの日本語学習済みパラメータのモデルです
model = BertModel.from_pretrained('bert-base-japanese-whole-word-masking')
model.eval()
print('ネットワーク設定完了')



ネットワーク設定完了


In [20]:
# BERTでベクトル化する関数を定義


def vectorize_with_bert(net, dataloader):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloader.batch_size

    # データローダーからミニバッチを取り出すループ
    for index, batch in enumerate(dataloader):
        # batchはTextとLableの辞書オブジェクト
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        inputs = batch.Text[0].to(device)  # 文章
        labels = batch.Label.to(device)  # ラベル

        # 順伝搬（forward）計算
        with torch.set_grad_enabled(False):

            # Berに入力
            result = net(inputs)

            # sequence_outputの先頭の単語ベクトルを抜き出す
            vec_0 = result[0]  # 最初の0がsequence_outputを示す
            vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
            vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換

            # ベクトル化したデータをtorchリストにまとめる
            if index == 0:
                list_text = vec_0
                list_label = labels
            else:
                list_text = torch.cat([list_text, vec_0], dim=0)
                list_label = torch.cat([list_label, labels], dim=0)

    return list_text, list_label


In [21]:
# DataLoaderをベクトル化版に変換
# 少し時間がかかります5分弱

list_text_train, list_label_train = vectorize_with_bert(model, dl_train)
list_text_eval, list_label_eval = vectorize_with_bert(model, dl_eval)
list_text_test, list_label_test = vectorize_with_bert(model, dl_test)


使用デバイス： cuda:0
-----start-------
使用デバイス： cuda:0
-----start-------
使用デバイス： cuda:0
-----start-------


In [22]:
# torchのリストをDatasetに変換

from torch.utils.data import TensorDataset

dataset_bert_train = TensorDataset(
    list_label_train.view(-1, 1), list_text_train)
dataset_bert_eval = TensorDataset(list_label_eval.view(-1, 1), list_text_eval)
dataset_bert_test = TensorDataset(list_label_test.view(-1, 1), list_text_test)


In [23]:
# Dataloaderにする
from torch.utils.data import DataLoader

batch_size = 1024

dl_bert_train = DataLoader(
    dataset_bert_train, batch_size=batch_size, shuffle=True, drop_last=True)
# drop_lastは最後のミニバッチがbatch_sizeに足りない場合は無視する

dl_bert_eval = DataLoader(
    dataset_bert_eval, batch_size=batch_size, shuffle=False)
dl_bert_test = DataLoader(
    dataset_bert_test, batch_size=batch_size, shuffle=False)


## 準備3：IICのディープラーニングモデルを用意

In [24]:
import torch.nn as nn
import torch.nn.functional as F

OVER_CLUSTRING_RATE = 10


class NetIIC(nn.Module):
    def __init__(self):
        super(NetIIC, self).__init__()

        # multi-headは今回しない
        self.conv1 = nn.Conv1d(1, 400, kernel_size=768, stride=1, padding=0)
        self.bn1 = nn.BatchNorm1d(400)
        self.conv2 = nn.Conv1d(1, 300, kernel_size=400, stride=1, padding=0)
        self.bn2 = nn.BatchNorm1d(300)
        self.conv3 = nn.Conv1d(1, 300, kernel_size=300, stride=1, padding=0)
        self.bn3 = nn.BatchNorm1d(300)

        self.fc1 = nn.Linear(300, 250)
        self.bnfc1 = nn.BatchNorm1d(250)

        # livedoorニュースの9カテゴリに対応するかな？と期待する9分類
        self.fc2 = nn.Linear(250, 9)

        # overclustering
        # 実際の想定よりも多めにクラスタリングさせることで、ネットワークで微細な変化を捉えられるようにする
        self.fc2_overclustering = nn.Linear(250, 9*OVER_CLUSTRING_RATE)

    def forward(self, x):
        x = x.view(x.size(0), 1, -1)
        x = F.relu(self.bn1(self.conv1(x)))

        x = x.view(x.size(0), 1, -1)
        x = F.relu(self.bn2(self.conv2(x)))

        x = x.view(x.size(0), 1, -1)
        x = F.relu(self.bn3(self.conv3(x)))

        x = x.view(x.size(0), -1)
        x_prefinal = F.relu(self.bnfc1(self.fc1(x)))

        # multi-headは使わず
        y = F.softmax(self.fc2(x_prefinal), dim=1)
        y_overclustering = F.softmax(self.fc2_overclustering(
            x_prefinal), dim=1)  # overclustering

        return y, y_overclustering


In [25]:
import torch.nn.init as init


def weight_init(m):
    """重み初期化"""
    if isinstance(m, nn.Conv1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm1d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        # Xavier
        # init.xavier_normal_(m.weight.data)

        # He
        init.kaiming_normal_(m.weight.data)

        if m.bias is not None:
            init.normal_(m.bias.data)


In [26]:
# IISによる損失関数の定義
# 参考：https://github.com/RuABraun/phone-clustering/blob/master/mnist_basic.py
import sys


def compute_joint(x_out, x_tf_out):
    bn, k = x_out.size()
    assert (x_tf_out.size(0) == bn and x_tf_out.size(1) == k), '{} {} {} {}'.format(
        bn, k, x_tf_out.size(0), x_tf_out.size(1))

    p_i_j = x_out.unsqueeze(2) * x_tf_out.unsqueeze(1)  # bn, k, k
    p_i_j = p_i_j.sum(dim=0)  # k, k
    p_i_j = (p_i_j + p_i_j.t()) / 2.  # symmetrise
    p_i_j = p_i_j / p_i_j.sum()  # normalise
    return p_i_j


def IID_loss(x_out, x_tf_out, EPS=sys.float_info.epsilon):
    # has had softmax applied
    bs, k = x_out.size()
    p_i_j = compute_joint(x_out, x_tf_out)
    assert (p_i_j.size() == (k, k))

    p_i = p_i_j.sum(dim=1).view(k, 1).expand(k, k)
    p_j = p_i_j.sum(dim=0).view(1, k).expand(k, k)

    # avoid NaN losses. Effect will get cancelled out by p_i_j tiny anyway
    # これはPyTorchのバージョン1.3以上だとエラーになる
    # https://discuss.pytorch.org/t/pytorch-1-3-showing-an-error-perhaps-for-loss-computed-from-paired-outputs/68790/3
    #p_i_j[(p_i_j < EPS).data] = EPS
    #p_j[(p_j < EPS).data] = EPS
    #p_i[(p_i < EPS).data] = EPS

    p_i_j = torch.where(p_i_j < EPS, torch.tensor(
        [EPS], device=p_i_j.device), p_i_j)
    p_j = torch.where(p_j < EPS, torch.tensor([EPS], device=p_j.device), p_j)
    p_i = torch.where(p_i < EPS, torch.tensor([EPS], device=p_i.device), p_i)

    # https://qiita.com/Amanokawa/items/0aa24bc396dd88fb7d2a
    # 参考に、重みalphaを追加

    alpha = 2.0
    loss = (- p_i_j * (torch.log(p_i_j) - alpha *
                       torch.log(p_j) - alpha*torch.log(p_i))).sum()

    return loss


In [27]:
# データにノイズを加える関数の定義
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tensor_std = list_text_train.std(dim=0).to(device)


def perturb_data(x):
    y = x.clone()
    noise = torch.randn(len(tensor_std)).to(device)*tensor_std*2.0
    noise = noise.expand(x.shape[0], -1)
    y += noise

    return y


## 4：IICのネットワークを学習させる

In [28]:
# 学習関数の定義

def train(total_epoch, model, train_loader, optimizer, device):

    # ネットワークを訓練モードに
    model.train()

    # 学習率のスケジューラーCosAnnealing
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=2, T_mult=2, eta_min=0)

    for epoch in range(total_epoch):
        for batch_idx, (target, data) in enumerate(train_loader):

            # 学習率変化
            scheduler.step()
            
            data_perturb = perturb_data(data)  # ノイズを与え、変換したデータを作る

            # GPUに送れる場合は送る
            data = data.to(device)
            data_perturb = data_perturb.to(device)

            # 最適化関数の初期化
            optimizer.zero_grad()

            # ニューラルネットワークへ入れる
            output, output_overclustering = model(data)
            output_perturb, output_perturb_overclustering = model(data_perturb)

            # 損失の計算
            loss1 = IID_loss(output, output_perturb)
            loss2 = IID_loss(output_overclustering,
                             output_perturb_overclustering)
            loss = loss1 + loss2

            # 損失を減らすように更新
            loss.backward()
            optimizer.step()

        # ログ出力
        if epoch % 50 == 0:
            print('Train Epoch {} \tLoss1: {:.6f} \tLoss2: {:.6f} \tLoss_total: {:.6f}'.format(
                epoch, loss1.item(), loss2.item(), loss1.item()+loss2.item()))

    return model, optimizer


In [29]:
# 学習の実施(5分弱)

# モデルの用意
net = NetIIC()
net.apply(weight_init)
net.to(device)

# 最適化関数
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4) 

total_epoch = 1000

model_trained, optimizer = train(
    total_epoch, net, dl_bert_train, optimizer, device)


Train Epoch 0 	Loss1: -3.945977 	Loss2: -7.733891 	Loss_total: -11.679867
Train Epoch 50 	Loss1: -6.163636 	Loss2: -10.227787 	Loss_total: -16.391423
Train Epoch 100 	Loss1: -6.519540 	Loss2: -11.245013 	Loss_total: -17.764553
Train Epoch 150 	Loss1: -6.554135 	Loss2: -11.682366 	Loss_total: -18.236502
Train Epoch 200 	Loss1: -6.559808 	Loss2: -12.294264 	Loss_total: -18.854072
Train Epoch 250 	Loss1: -6.540496 	Loss2: -12.461839 	Loss_total: -19.002335
Train Epoch 300 	Loss1: -6.548514 	Loss2: -13.140853 	Loss_total: -19.689367
Train Epoch 350 	Loss1: -6.542245 	Loss2: -13.114568 	Loss_total: -19.656813
Train Epoch 400 	Loss1: -6.576515 	Loss2: -13.155249 	Loss_total: -19.731764
Train Epoch 450 	Loss1: -6.554772 	Loss2: -13.119078 	Loss_total: -19.673850
Train Epoch 500 	Loss1: -6.567766 	Loss2: -13.148041 	Loss_total: -19.715806
Train Epoch 550 	Loss1: -6.571950 	Loss2: -13.201900 	Loss_total: -19.773850
Train Epoch 600 	Loss1: -6.545648 	Loss2: -13.169672 	Loss_total: -19.715320
Tra

In [30]:
# モデル分類のクラスターの結果を確認する
import numpy as np

# ミニバッチサイズ1のテスト用のDataLoaderを用意
dl_bert_test = DataLoader(
    dataset_bert_test, batch_size=1, shuffle=False)


def test(model, device, test_loader):
    model.eval()

    out_targs = []
    ref_targs = []

    # 出力用のリストを用意
    total_num = len(test_loader)
    # index, (target_label, inferenced_label)
    output_list = np.zeros((total_num, 2))

    with torch.no_grad():
        for batch_idx, (target, data) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device)
            outputs, outputs_overclustering = model(data)

            # 分類結果をリストに追加
            out_targs.append(outputs.argmax(dim=1).cpu())
            ref_targs.append(target[0].cpu())

            # 結果をリストにまとめる
            output_list[batch_idx, 0] = target[0][0].cpu()  # 正解ラベル
            output_list[batch_idx, 1] = outputs.argmax(dim=1).cpu()  # 予測ラベル

    out_targs = torch.cat(out_targs)
    ref_targs = torch.cat(ref_targs)

    return out_targs.view(-1, 1).numpy(), ref_targs.numpy(), output_list


In [31]:
# テストデータで推論を実施
out_targs, ref_targs, output_list = test(model_trained, device, dl_bert_test)


In [32]:
# 混同行列（的な）を作る
matrix = np.zeros((9, 9))

# 縦にlivedoorニュースの正解クラスを、横に判定されたクラスの頻度表を作成
for i in range(len(out_targs)):
    row = ref_targs[i]
    col = out_targs[i]
    matrix[row][col] += 1

np.set_printoptions(suppress=True)
print(matrix)


[[  7.   6.  15.   1.  36.   8.  13.   4.  16.]
 [ 25.   0.  63.  25.  18.  19.  28.   4.   3.]
 [ 26.   1.   0.   0.   2.  44.   1.  75.   2.]
 [  4. 128.   1.   2.   5.   3.   8.  11.   2.]
 [  7.   0.   5.  93.   7.  13.  37.   0.   2.]
 [ 58.   1.   0.   0.   5.  53.   1.  62.   1.]
 [  3.   6.   1.   0.  16.   1.   1.   4. 140.]
 [ 13.  13.  12.   1.  48.   9.  49.   4.  26.]
 [ 58.   3.  21.   8.  12.  46.  20.   3.   6.]]


In [33]:
dic_id2cat


{0: 'livedoor-homme',
 1: 'it-life-hack',
 2: 'topic-news',
 3: 'movie-enter',
 4: 'smax',
 5: 'sports-watch',
 6: 'dokujo-tsushin',
 7: 'peachy',
 8: 'kaden-channel'}

In [34]:
# クラスタの結果を確認
#「sports-watch」の5番目のクラスタの文章、7番目のクラスタの文章
#「topic-news」の5番目のクラスタの文章、7番目のクラスタの文章
# を確認して、クラスタ5とクラスタ7の特徴を見てみます。

import pandas as pd

df2 = pd.DataFrame(output_list)
df2.columns=["正解クラス", "推定クラスタ"]
df2.head()

,正解クラス,推定クラスタ
0,5.0,5.0
1,6.0,8.0
2,3.0,6.0
3,6.0,8.0
4,5.0,7.0


In [35]:
df2[(df2['正解クラス']==0) & (df2['推定クラスタ']==5)].head()

,正解クラス,推定クラスタ
29,0.0,5.0
568,0.0,5.0
678,0.0,5.0
812,0.0,5.0
1046,0.0,5.0


In [36]:
# dfに元文書を入れている。300文字ほど見る。
print(df.iloc[21, 0][:300])
print(df.iloc[59, 0][:300])


こんにちは、「ビズリーチ年収1000万円研究所」所長の佐藤和男です。この研究所では、年収1000万円以上のビジネスパーソンに対してさまざまなアンケートを取り、“年収1000万円を稼ぐビジネスパーソンの考え方”を調査しています。今回のご報告は、経験豊富なビジネスパーソンが就職活動中の大学生にお勧めしたい「本当の企業ランキング」。長引く不況で就職氷河期と言われる昨今、就職するならぜひ目指して欲しいと思う企業のアンケートを行いました。年収1000万円のビジネスパーソンは、どんな企業を選ぶのか——。共通する傾向を探りました。Q1.就職活動中の大学生にお勧めしたい就職先企業を3社まで挙げてください。1位
日常の何気ない気持ちをTwitterにつぶやいたり、実名登録のFacebookで懐かしい友人と再会したり、SNSはもはや我々の生活において欠かせない存在となりつつある。先日、国内の月間利用者数が1,000万人を突破し、mixi（1,520万人、2011年12月現在）を追い抜くのも時間の問題と思われるFacebookでは、診断やゲームなど様々なアプリが生まれ、ユーザーのタイムラインを今日も賑わしている。しかし、その一方で、Facebookを悪用するケースもまた徐々に増え始めている。Facebookでは、2008年1月にAPIが公開されて以来、様々なアプリが誕生しているが、同年8月にはボット型の不


In [37]:
df2[(df2['正解クラス']==0) & (df2['推定クラスタ']==7)].head(2)

,正解クラス,推定クラスタ
185,0.0,7.0
495,0.0,7.0


In [38]:
# dfに元文書を入れている。300文字ほど見る。
print(df.iloc[14, 0][:300])
print(df.iloc[18, 0][:300])

◆プロフィール小沢コージバブル期にソアラ、プレリュードを横目で眺め、トレンディドラマを見、合コンしまくったスーパーカーブームど真ん中イケメン自動車ジャーナリスト。現在『BESTCAR』『ENGINE』『DIME』誌ほか『webCG』『日経TRENDY』『carview』などwebでも大活躍！吉田由美当時、ミスコン荒らしとして20コ以上のミスの肩書きを持ち、某専門誌「○×のすべて」に助手席モデルとして君臨。「経歴は編集長より長いかも（笑）」（本人談）という業界きっての謎の美女。今はカーライフエッセイストの名のもと「ブログの女王」として自動車メディアはもちろんファッション誌に無くてはならない存在と
現在はMacRumors（Macのうわさ）というサイトでしか見ることができないが、6日ほど前にKitGuruというサイトが中国のiPhoneケースベンダーに持ち込まれたという「iPhone 5のモックアップ」と称する写真を掲載していた（ちなみに現在はどこからかの強い圧力のため削除されている模様）。このモックアップがホンモノであることを証明するかのように、楽天市場などのショッピングサイトで「iPhone 5用ケース先行販売」という形でケースが販売されている。「iPhone 5 ケース」での検索結果検索された商品のページ。何と購入可能。試しに楽天で「iPhone 5 ケース」というキーワードで検索


In [39]:
df2[(df2['正解クラス']==8) & (df2['推定クラスタ']==5)].head(2)

,正解クラス,推定クラスタ
55,8.0,5.0
65,8.0,5.0


In [40]:
# dfに元文書を入れている。300文字ほど見る。
print(df.iloc[7, 0][:300])
print(df.iloc[55, 0][:300])

AKB48の篠田麻里子のツイッターが人気だ。じゃんけん大会で優勝しセンターを射止めて以来さらに注目されている彼女。ツイッターでメンバーからもらったお土産を紹介し話題になっている。写真付きで紹介されたツイートはこちら。ちなみにニャロから敦子へのお土産は コチラ！！！！ 『あっちゃんに似てるー(((бвб))』 って言ってたよ（笑） ……(´∵`)ノあっちゃんもびっくりだ〜一緒に添付されている画像は、確かに前田敦子に似ていると言えなくもない女性の裸体をモチーフにしたお土産。そして、そのお土産よりも衝撃なのは篠田麻里子が前田敦子に選んだ顔文字だ。以前より、前田敦子はタモリなどにも番組で顔のパーツが中
NHN Japanは、同社が提供する「livedoorニュース」で配信している記事の中から、IT記事に特化したニュースをまとめて閲覧することができるスマートフォンアプリ「ITニュース by livedoor ニュース」（iOS・Android対応／無料）を公開している。好きなジャンルの情報を逃さない向かうところ敵なしのITニュースリーダーだ。というのも、このアプリではlivedoorニュースの編集部が厳選したIT記事が提供されるが、編集部の手作業によって「ITビジネス」「WEBサービス」「マーケティング」「モバイル」「デジタル家電」などの12のカテゴリに振り分けられるので、数ある情報の中から興


In [41]:
df2[(df2['正解クラス']==8) & (df2['推定クラスタ']==7)].head(2)

,正解クラス,推定クラスタ
546,8.0,7.0
692,8.0,7.0


In [42]:
# dfに元文書を入れている。300文字ほど見る。
print(df.iloc[71, 0][:300])
print(df.iloc[76, 0][:300])

フィクションの中だけだと思われていた技術が、次々と現実化している。MITメディアラボの研究グループが開発に成功したのは、何と曲がり角の先を撮影することのできるカメラだ。光ファイバーを伸ばして曲がり角の先の様子を確認することなら今の技術でもできるわけだが、MITメディアラボが開発したのはそういうものではない。曲がり角から離れた位置、つまり角を曲がる前に先の様子を撮影できるのだ。この技術のポイントは、レーザーを使うことにある。カメラに設置されたレーザー発振装置からレーザーパルスが発射され、カメラから見える位置にある壁にぶつかる。レーザー光は、壁にぶつかって飛び散り、いくつかはカメラから見えない位置
ＮＨＫは４月に民放大手５社と電通が進めているインターネットテレビ事業へ参入するという方針を固めた。これは、通常放送のほかに過去の番組約１万本を視聴できるようにするというものだ。深刻化するテレビ離れを食い止める狙いだという。過去番組１万本のうち、NHKが所有するのは４０００本程度になるとみられる。ネット上では「NHKも参加するんだ」という驚きの声や「おもしろい番組があれば見る」といったコメントが寄せられている。もちろんラインナップや仕組みにもよるが、過去のものを豊富に見ることができる環境ができることで今までとは違ったテレビ生活が生まれそうだ。ＮＨＫ、ネットテレビ参入４月、民放計画に合流■関連記事


文章を見ただけでは、あまり特徴は分かりませんね。。。

ただ、スポーツとニュースは、記事の雰囲気がとても似ていることは分かります。

これ以上に、IICされたクラスタの特徴をきちんと把握するには、

- wordcloudで単語の頻度の傾向を見てみる
- クラスタ文書の全ベクトルをいじいじして、なんらかクラスタのベクトルを作って、代表文書を決めたり近い単語を出す

などの操作が考えられます。

以上。